# Reducing Concussions on Punt Plays
by Charles Belina and George Ormseth

## I. Data Input & Manipulation

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from collections import Counter

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
# Here I input the revelant data using Pandas.
vid_rev_DF = pd.read_csv('../input/video_review.csv')
player_data_DF = pd.read_csv('../input/play_player_role_data.csv')
play_info_DF = pd.read_csv('../input/play_information.csv')
player_punt_DF = pd.read_csv('../input/player_punt_data.csv')

In [ ]:
# Quick test to see what the data looks like.
vid_rev_DF.tail()

In [ ]:
# Here I begin to store all of the revelant information in arrays.
Concussed_position = []; # the position that a concussed player was playing on the punt play
C_YdLines = []; # the yard line the ball was snapped from on the punt play
C_Punting_Team = []; # 
traditional_C_pos = [];
player_number = [];
player_team = [];
GameIDs = np.array(vid_rev_DF["GameKey"].values);
PlayIDs = np.array(vid_rev_DF["PlayID"].values);
GSISIDs = np.array(vid_rev_DF["GSISID"].values);


Guard_C_plays=[];

for i in range(len(GameIDs)):
    Game = GameIDs[i];
    Play = PlayIDs[i];
    Player = GSISIDs[i];
    
    Concussed_player_trad_pos = player_punt_DF.loc[(player_punt_DF['GSISID'] == Player)];
    traditional_C_pos.append(Concussed_player_trad_pos.iloc[0]['Position']);
    player_number.append(Concussed_player_trad_pos.iloc[0]['Number']);
    
   # if "d" not in Concussed_player_trad_pos.iloc[0]['Number']:
   #     player_team.append
        
   # else: 
    
    Concussion_play = play_info_DF.loc[(play_info_DF['PlayID'] == Play)
                                         & (play_info_DF['GameKey'] == Game)];
    C_YdLines.append(Concussion_play.iloc[0]['YardLine']);
    C_Punting_Team.append(Concussion_play.iloc[0]['Poss_Team']);
    
    Concussed_player = player_data_DF.loc[(player_data_DF['PlayID'] == Play)
                                         & (player_data_DF['GameKey'] == Game)
                                        & (player_data_DF['GSISID'] == Player)];
    #print(Concussed_player);
    Concussed_position.append(Concussed_player.iloc[0]['Role']);
    
    if ((Concussed_player.iloc[0]['Role']=='PLG') or Concussed_player.iloc[0]['Role']=='PRG'):
        Guard_C_plays.append(Concussed_player.iloc[0]['PlayID'])
        #Guard_C_


## II. Data Analysis
### II.I Concussed Position.
Lets begin by looking at which players are being concussed the most.

In [ ]:
String_counts1 = Counter(Concussed_position)

#frequencies = String_counts.values()
#names = String_counts.keys()

#x_coordinates = np.arange(len(String_counts))
#fig = plt.figure()
#ax = fig.add_subplot(111)
#ax.bar(x_coordinates, frequencies, align='center')
#ax.xaxis.set_major_locator(plt.FixedLocator(x_coordinates))
#ax.xaxis.set_major_formatter(plt.FixedFormatter(names))
#plt.show()
df = pd.DataFrame.from_dict(String_counts1, orient='index')
plot = df.plot(kind='bar', figsize = (9, 6),
               title="Concussed Player Position on Punt Return", legend=False,
                fontsize = 18)
plot.title.set_size(24);
plot.set_xlabel("Player Position", fontsize=20);
plot.set_ylabel("Number of Concussions", fontsize= 20);

The above figure shows the number of concussions sustained per position during a punt return. Unsurprisingly, punt returners have suffered the most concussions over the past two years on punt plays; however, the x-axis contains too many positions to glean much more. Let's use a dictionary to group similiar positions under one heading. For example, "PLT" and "PRT" will now be grouped together under the title "Tackle."

In [ ]:
Punt_Positions = {
    # Punt Coverage Positions
    'PLG': 'Guard',
    'PRG': 'Guard',
    'PRT': 'Tackle',
    'PLT': 'Tackle',
    'GR': 'Gunner',
    'GL': 'Gunner',
    'PRW': 'Wing',
    'PLW': 'Wing',
    'PLS': 'Center',
    'P': 'Punter',
    'PPR': 'Protector',
    
    # Punt Return Positions
    'PR': 'Returner',
    'PDR1': 'DLine',
    'PDR2': 'DLine',
    'PDR3': 'DLine',
    'PDL1': 'DLine',
    'PDL2': 'DLine',
    'PDL3': 'DLine',
    'PLR': 'Linebacker',
    'PLM': 'Linebacker',
    'PLL': 'Linebacker',
    'VR': 'Cornerback',
    'PFB': 'Ret Protector'
}

Punt_Cov = 0;
Punt_Ret = 0;

In [ ]:
Concussed_position2 = [Punt_Positions[k] for k in Concussed_position]

for i in range(len(Concussed_position2)):
    pos = Concussed_position2[i];
    if ((pos == 'Guard') or (pos == 'Tackle') or (pos == 'Gunner') or (pos == 'Wing') or (pos == 'Center')
       or (pos == 'Punter') or (pos == 'Protector')):
        Punt_Cov += 1;
    else: Punt_Ret += 1;

In [ ]:
String_counts2 = Counter(Concussed_position2)

#frequencies = String_counts.values()
#names = String_counts.keys()

#x_coordinates = np.arange(len(String_counts))
#fig = plt.figure()
#ax = fig.add_subplot(111)
#ax.bar(x_coordinates, frequencies, align='center')
#ax.xaxis.set_major_locator(plt.FixedLocator(x_coordinates))
#ax.xaxis.set_major_formatter(plt.FixedFormatter(names))
#plt.show()
df = pd.DataFrame.from_dict(String_counts2, orient='index')
plot = df.plot(kind='bar', figsize = (9, 6),
               title="Concussed Player Position on Punt Return", legend=False,
                fontsize = 12)
plot.title.set_size(24);
plot.set_xlabel("Player Position", fontsize=20);
plot.set_ylabel("Number of Concussions", fontsize= 20);

After seeing this, we were surprised at the number of guards being concussed. It was surprising because (i) guards tend to be slower players and (ii) they on the coverage team, not the return team. We then decided to look which team (return or coverage) was sustaining the most concussions. 

In [ ]:
plt.pie([Punt_Cov, Punt_Ret], explode=(0.1, 0), labels=('Coverage Team', 'Return Team'),
        autopct='%1.1f%%', shadow=True, startangle=180);
plt.axis('equal');
plt.show()

The fact that the coverage team sustains nearly 3/4 of concussions was very surprising and left us ready to watch the videos to see why coverage players were being injured at a much higher clip.

### II.II Video Analysis
After determining that (i) guards are sustaining a disproportionate number of concussions and that (ii) players on the coverage team were suffering a majority of the cvoncussions, we began to watch the videos of the plays which had resulted in concussions.

The videos immediately showed that guards (and oftentimes other players sustaining concussions) were in fact injuring themselves. In attempts to tackle or block an oppenent, they would leave their feet and launch in situations where they could have come to balance before making the tackle or attempting a block.

## III. Rule Change
With this in mind, we propose to **eliminate any form of launching on punt plays.** Currently, the NFL states that launching is only a penalty if it is done to a defenseless player. We believe the NFL should expand this rule to apply to all players, *whether defenseless or not*, during a punt play.

After a player leaves his feet, he no longer has control over what he will hit. On punts, this becomes especially dangerous because the play is faster than a standard play and much can change after a player has launched.

After watching each concussion video many times, we estimated that **32.4% of concussions could have been prevented if a player had not launched.** This amounts to 12 of the 37 concussions.


In [ ]:
plt.pie([12, 25], explode=(0.1, 0), labels=('Launch Plays', 'Other'),
        autopct='%1.1f%%', shadow=True, startangle=180);
plt.axis('equal');
plt.show()

For reference, the Game and Play IDs of the concussion plays that were caused by a launch are given below.

In [ ]:
C_plays = {'Game ID': [5, 29, 189, 231, 234, 281, 296, 357, 384, 506, 553, 618],
     'Play ID': [3129, 538, 3509, 1976, 3278, 1521, 2667, 3630, 183, 1988, 1683, 2792]}
C_plays_df = pd.DataFrame(data=C_plays)
C_plays_df

## IV. Conclusion

After seeing that coverage players were injuring themselves at a much higher clip than return players, we turned to the videos and understood that this was happening because coverage players tend to launch and inadvertently injure themselves. Under the current NFL rules, launching is only illegal if it is done to a defenseless player. We imagine that this was instituted to protect the defenseless player, but on punt plays, it is oftentimes the launcher who is injured.

In order to protect tacklers and blockers from concussions, they should be discouraged from launching in any way. We thus propose **eliminating any form of launching** and believe that this could lead to a **30% reduction in concussions on punt plays**. 